# A-Hard-Problem

### import necessary libraries

In [1]:
!pip install vaderSentiment
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import sklearn as sklearn
import gensim as gensim
import keras as keras
import tensorflow as tf
import nltk
import time
import re
import twint
import json
import gensim.corpora as corpora
import spacy
import pyLDAvis
import pyLDAvis.gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pprint import pprint
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sullivanmyer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sullivanmyer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sullivanmyer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# read in csv files and concatenate
df1 = pd.read_csv('tweet_nlp/just_cv/twint_3_en.csv')
df2 = pd.read_csv('tweet_nlp/just_cv/twint_4_en.csv')
df3 = pd.read_csv('tweet_nlp/just_cv/twint_5_en.csv')
df4 = pd.read_csv('tweet_nlp/just_cv/twint_6_en.csv')
df5 = pd.read_csv('tweet_nlp/just_cv/twint_7_en.csv')
df6 = pd.read_csv('tweet_nlp/just_cv/twint_8_en.csv')
df = pd.concat([df1,df2,df3,df4,df5,df6])

In [3]:
# preliminary look at our data
print(df.shape)
df.head()

(85716, 34)


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1242886541418037251,1241512518721376257,1585162217000,2020-03-25,14:50:17,EDT,2758562689,adrisml777,Adrian Finn George,NaN,"I said they are better than nothing, not that ...","['mmmbiata', 'iatemuggles']",[],[],0,0,0,[],[],https://twitter.com/AdrisML777/status/12428865...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2758562689', 'username': 'AdrisM...",NaN,NaN,NaN,NaN
1,1242886541271273473,1242886541271273473,1585162217000,2020-03-25,14:50:17,EDT,1212444154761863168,sammendment,Second Amendment,NaN,Oops: CNN Forgot To Translate Its Latest Coron...,[],['https://babylonbee.com/news/cnn-embarassed-a...,[],0,0,0,[],[],https://twitter.com/SAmmendment/status/1242886...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1212444154761863168', 'username'...",NaN,NaN,NaN,NaN
2,1242886540839305216,1242886540839305216,1585162217000,2020-03-25,14:50:17,EDT,939935680137920515,still_a_nerd,the coronavirus movie,NaN,Speeding up https://twitter.com/ryanfiredragon...,[],['https://twitter.com/ryanfiredragon/status/12...,[],0,0,0,[],[],https://twitter.com/still_a_nerd/status/124288...,False,https://twitter.com/ryanfiredragon/status/1242...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '939935680137920515', 'username':...",NaN,NaN,NaN,NaN
3,1242886539786350592,1242886539786350592,1585162217000,2020-03-25,14:50:17,EDT,144702285,sebasaherrera,Sebastian Herrera,NaN,"Texas Gov. Greg Abbott, who routinely overrule...","['efindell', 'wsj']",['https://www.wsj.com/articles/texas-coronavir...,[],0,0,0,[],[],https://twitter.com/SebasAHerrera/status/12428...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '144702285', 'username': 'SebasAH...",NaN,NaN,NaN,NaN
4,1242886539564257292,1242834296358014977,1585162217000,2020-03-25,14:50:17,EDT,1418644652,rkelly1972a,Robert Kelly,NaN,"Other than Satan, is there anyone more evil th...","['seanhannity', 'speakerpelosi']",[],['https://pbs.twimg.com/media/ET-euAMWsAEZ7nP....,0,0,0,"['#stimulusplan', '#covid2019', '#swampcreatur...",[],https://twitter.com/rkelly1972a/status/1242886...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1418644652', 'username': 'rkelly...",NaN,NaN,NaN,NaN


In [23]:
# drop unnecessary columns
data = pd.DataFrame(df['tweet'])
len(data)

85716

In [10]:
# instantiate a VADER sentiment analyzer
analyser = SentimentIntensityAnalyzer()

In [24]:
data.dropna(subset = ['tweet'], inplace=True)

In [25]:
non_string = []
for i in df.tweet:
    if type(i) is not str:
        non_string.append(i)


In [26]:
len(data)

85715

In [ ]:
# get sentiment scores, store compound sentiment score as our sentiment column
data['sentiment'] = [analyser.polarity_scores(i) for i in data['tweet']]
data['compound_sentiment'] = [i['compound'] for i in df['sentiment']]
data['sentiment'] = df['compound_sentiment']
data.drop(columns=['compound_sentiment'], inplace=True)

In [27]:
data.head()

,tweet
0,"I said they are better than nothing, not that ..."
1,Oops: CNN Forgot To Translate Its Latest Coron...
2,Speeding up https://twitter.com/ryanfiredragon...
3,"Texas Gov. Greg Abbott, who routinely overrule..."
4,"Other than Satan, is there anyone more evil th..."


In [ ]:
#view sentiment distribution
df.sentiment.describe()

In [ ]:
sns.distplot(df.sentiment)

In [28]:
#tokenize tweets
data['tweet_vec'] = data['tweet'].map(nltk.word_tokenize)

In [29]:
data.head()

,tweet,tweet_vec
0,"I said they are better than nothing, not that ...","[I, said, they, are, better, than, nothing, ,,..."
1,Oops: CNN Forgot To Translate Its Latest Coron...,"[Oops, :, CNN, Forgot, To, Translate, Its, Lat..."
2,Speeding up https://twitter.com/ryanfiredragon...,"[Speeding, up, https, :, //twitter.com/ryanfir..."
3,"Texas Gov. Greg Abbott, who routinely overrule...","[Texas, Gov, ., Greg, Abbott, ,, who, routinel..."
4,"Other than Satan, is there anyone more evil th...","[Other, than, Satan, ,, is, there, anyone, mor..."


In [30]:
data = data['tweet_vec']
data

0        [I, said, they, are, better, than, nothing, ,,...
1        [Oops, :, CNN, Forgot, To, Translate, Its, Lat...
2        [Speeding, up, https, :, //twitter.com/ryanfir...
3        [Texas, Gov, ., Greg, Abbott, ,, who, routinel...
4        [Other, than, Satan, ,, is, there, anyone, mor...
                               ...                        
13714    [RT, to, tell, ⁦, @, USAID⁩, to, stop, aid, cu...
13715    [For, those, of, you, who, 've, been, followin...
13716    [Maybe, ,, his, death, will, save, the, lives,...
13717    [y, ’, all, go, watch, this, corona, virus, mo...
13718    [@, USDOL, just, released, this, to, workplace...
Name: tweet_vec, Length: 85715, dtype: object

In [31]:
#instantiate a w2v model
w2v_model = gensim.models.Word2Vec(data, size=100, window=5, min_count=1, workers=4)

In [32]:
#train the w2v model
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=100)

(189612895, 247008700)

In [33]:
#get what matters (weights, etc..), throw away the actual model to reclaim memory space
word_vectors = w2v_model.wv
del w2v_model

In [61]:
word_vectors.most_similar(positive=['president'])

[('President', 0.6168984174728394),
 ('administration', 0.5924678444862366),
 ('government', 0.5718132257461548),
 ('he', 0.5284136533737183),
 ('realDonaldTrump', 0.5188762545585632),
 ('presidency', 0.5158246755599976),
 ('leader', 0.5008671283721924),
 ('ego', 0.49674510955810547),
 ('bigot', 0.4958049952983856),
 ('UAZMedTucson', 0.48750990629196167)]

In [ ]:
#word_vectors.accuracy('questions-words.txt')

In [ ]:
all_words = ' '.join(df['SentimentText'].values)

In [ ]:
all_words = all_words.split(' ')

In [ ]:
all_words.remove('')

In [ ]:
all_words_dict = {}
for i in all_words:
    if i not in all_words_dict.keys():
        all_words_dict[i] = 1
    else:
        all_words_dict[i] += 1

In [ ]:
sorted_words = sorted(all_words_dict, key=all_words_dict.get, reverse=True)
sorted_words.remove('')

In [ ]:
stop_words = list(set(nltk.corpus.stopwords.words('english')))

In [ ]:
more_stop = ['and',
             'of',
             'in',
             'he',
             'was',
             'at',
             'not',
             'as',
             'it',
             'I',
             'but',
             'she',
             'The',
             'said',
             'were',
             'be',
             'which',
             'who',
             'one',
             'He',
             'an',
             'he',
             'as',
             'she',
             'be',
             'He',
             'Planet',
             'eBooks',
             'Free',
             'eBook.com',
             'could',
             'no',
             'will',
             'now',
             'went',
             'But',
             'And',
             'him.',
             'man',
             '‘I',
             'old',
             'him,',
             'She',
             'would',
             'eBook.com',
             'him.',
             'him,',
             'like',
             'came',
             'looked',
             'time',
             'began',
             'know',
             'without',
             'face',
             'thought',
             'felt',
             'see',
             'still',
             'ing',
             'men',
             'It',
             'seemed',
             'left',
             'go',
             'little',
             'come',
             'In',
             'eyes',
             'two',
             'even',
             'must',
             'long',
             'saw',
             'made',
             'something',
             'go',
             'come',
             'always',
             'it.',
             'heard',
             'say',
             'army',
             'and,',
             'young',
             'took',
             'told',
             'don’t',
             'back',
             'the',
             'a',
             'for',
             'of',
             'have',
             'me',
             'was',
             "I'm",
             'are',
             'with',
             '-',
             '_']

In [ ]:
stop_words = stop_words + more_stop

In [ ]:
more = ["news","case","make","would","death","spread","story","still","stop","kill","com", "twitter","pic","say","https"]
more2 = ["help","work","time","new","covid","get","test","day","http","country","good","world","may","government","pat"]
more3 = ["due","stay","be","report","worker","bill","back","first","people","corona"]
more4 = ["go","die","take","see","know","status","call",'Go','Say', 'www', 'many', 'see', 'make', 'want', 'be', 'could', "s", 'tell', 'today']
more5 = ['Let', 'Be', 'case', 'make', 'also', 'let']

In [ ]:
more.extend(more2)
more.extend(more3)
more.extend(more4)
more.extend(more5)

In [ ]:
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(more)

In [ ]:
data = df['SentimentText']

In [ ]:
types = []
for i in data:
    types.append(type(i))

In [ ]:
index = 0
non_str = {}
fake = []
for i in data: 
    if type(i) != str:
        non_str[index] = type(i)
    index += 1

In [ ]:
for i in non_str:
    data = np.delete(data, i)

In [ ]:
#filter out tweerts with fewer than 10 words
ndata = []
for i in data:
    if len(i.split(' ')) > 10:
        ndata.append(i)

In [ ]:
len(data), len(ndata)

In [ ]:
# create a text pre-processing pipeline
def corpus_pipeline(data):  
    # ********    THE FOLLOWING REGEXES SHOULD PROBABLY BE HEAVILY MODIFIED FOR CUSTOMIZING TWEETS CLEANING  *************
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # **********   ALSO PROBABLY REQUIRES EXTENSIVE ADDING OF TWEET-SPECIFIC STOPWORDS   ***********
    # Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    
    data_words_trigrams = make_trigrams(data_words_bigrams)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, texts, corpus

In [ ]:
# store results from our pipeline function call
id2word, texts, corpus = corpus_pipeline(ndata)

In [ ]:
# compute coherence values for various number of topics.
def compute_coherence_values(dictionary, corpus, texts, limit, start=5, step=5): 
    """ Compute coherence values for various number of topics.
    Parameters:
        dictionary : Gensim dictionary
        corpus : Gensim corpus
        texts : List of input texts
        limit : Max num of topics
        start : Int of lowest number of topics to model
        step : Int of increment to change number of topics to model
    Returns:
        model_list : List of LDA topic models
        coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        start=time.time()
        model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=1000,
                                           passes=1,
                                           per_word_topics=True)
                                            
        
        print(f'Topic modeling for {num_topics} topics took {time.time()-start} seconds.')
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
strt = 12
lmt = 17
stp = 1

In [ ]:
# store computed coherence values
ml, cov = compute_coherence_values(id2word, corpus, texts, lmt, strt, stp)

In [ ]:
def graph_coherence(coherence_values):
    """ Graph a list of coherence values to determine the optimal number of topics to model. """
    limit=lmt; start=strt; step=stp;
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [ ]:
graph_coherence(cov)

In [ ]:
ml[2]

In [ ]:
ml[-1]

In [ ]:
optimal_model = ml[-1]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

In [ ]:
# Build LDA model
lda_model_5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model_5.show_topics()

In [ ]:
# Build LDA model
lda_model_10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model_10.show_topics()

In [ ]:
## Save a dictionary into a pickle file.
#import pickle
#
#pickle.dump(lda_model_10, open("mode1_10_345", "wb" ))

In [ ]:
#import pickle
#lda_model_10 = pickle.load( open( "mode1_10_345", "rb" ) )
## favorite_color is now { "lion": "yellow", "kitty": "red" }

In [ ]:
optimal_model = lda_model_10
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_10, corpus, id2word)
vis

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_5, corpus, id2word)
vis

In [ ]:
regex = r"[a-zA-Z+]+"

topic_1_str = '0.067*\"thank\" + 0.030*\"feel\" + 0.030*\"look\" + 0.026*\"tweet\" + 0.025*\"good\" + 0.023*\"watch\" + 0.017*\"keep\" + 0.016*\"sleep\" + 0.014*\"make\" + 0.012*\"see\"'
topic_2_str = '0.068*"s" + 0.021*"fun" + 0.021*"wish" + 0.020*"find" + 0.016*"lot" + 0.015*"tomorrow" + 0.015*"morning" + 0.014*"give" + 0.014*"year" + 0.013*"weekend"'
topic_3_str = '0.077*"be" + 0.076*"not" + 0.039*"do" + 0.023*"get" + 0.022*"go" + 0.021*"love" + 0.018*"have" + 0.018*"can" + 0.017*"think" + 0.014*"really"'
topic_4_str = '0.058*"quot" + 0.035*"need" + 0.035*"lol" + 0.028*"try" + 0.026*"bad" + 0.017*"sound" + 0.012*"check" + 0.011*"enjoy" + 0.011*"hate" + 0.011*"day"'
topic_5_str = '0.034*"hope" + 0.031*"amp" + 0.026*"great" + 0.020*"last" + 0.018*"week" + 0.017*"sorry" + 0.016*"night" + 0.015*"follow" + 0.014*"mean" + 0.014*"start"'

matches_1 = re.finditer(regex, topic_1_str)
matches_2 = re.finditer(regex, topic_2_str)
matches_3 = re.finditer(regex, topic_3_str)
matches_4 = re.finditer(regex, topic_4_str)
matches_5 = re.finditer(regex, topic_5_str)

matches_list = [matches_1, matches_2, matches_3, matches_4, matches_5]

topic_dict = {}
count = 0
for i in matches_list:
    count += 1
    topic_list = []
    for j, match in enumerate(i, start=1):
        if j%2 != 0:
            topic_list.append(match.group())
    topic_dict.setdefault('topic_{}'.format(count), topic_list)
topic_dict

In [ ]:
for i in data

# Word2Vec-Keras DIY Sentimentizer

In [ ]:
for i in sorted_words:
    if i in stop_words:
        sorted_words.remove(i)

In [ ]:
sorted_words[:10]

In [ ]:
sorted_words_dict = {}
for i in sorted_words:
    sorted_words_dict[i] = all_words_dict[i]

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer() 

In [ ]:
lemmatized_list = []
for i in sorted_words:
    lemmatized_list.append(lemmatizer.lemmatize(i))

In [ ]:
lemmatized_list

In [ ]:
print('Number of unique words: {}'.format(len(sorted_words_dict)))
print(len('Number of lemmatized stems: {}'.format(lemmatized_list)))

In [ ]:
lemmatized_dict = {}
for i in lemmatized_list:
    if i in lemmatized_dict.keys():
        lemmatized_dict[i] += 1
    else:
        lemmatized_dict[i] = 1

In [ ]:
len(lemmatized_dict)

In [ ]:
embeddings = word_vectors.get_keras_embedding(train_embeddings=False)

In [ ]:
#model = models.Sequential()
#
#model.add(layers.embeddings.Embedding(input_dim=max_words,
#  output_dim=embed_vec_len, embeddings_initializer=e_init,
#  mask_zero=True))
#
## specify the first hidden layer
#model.add(Dense(50, activation='relu', input_shape=(784,)))
#
## specify the second layer
#model.add(Dense(50, activation='relu'))
#
## specify the output layer
#model.add(Dense(num_classes, activation='sigmoid'))
#
#model.summary()
#
#model.compile(loss='categorical_crossentropy',
#              optimizer=SGD(),
#              metrics=['accuracy'])
#
#history = model.fit(x_train, y_train,
#                    batch_size=batch_size,
#                    epochs=epochs,
#                    verbose=1,
#                    validation_data=(x_test, y_test))
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])